## The particle layer interface

We start by activating the IPython extension and importing and aliasing a few useful components. We also select the CKD mode.

In [ ]:
%load_ext eradiate
import eradiate
import numpy as np
import matplotlib.pyplot as plt
from eradiate.notebook.tutorials import plot_sigma_t

eradiate.set_mode("ckd")

In [ ]:
import eradiate.scenes as ertsc
from eradiate import unit_registry as ureg

In [ ]:
particle_layer_default = ertsc.atmosphere.ParticleLayer()
particle_layer_default

We see that our `particle_layer_default` variable represents an aerosol layer with:

* a vertical extent from 0 to 1 km;
* a uniform particle density distribution within that extent;
* a division into 16 uniform cells;
* an optical thickness at 550 nm equal to 0.2.

We can now visualise the extinction coefficient using the following convenience function:

In [ ]:
plot_sigma_t(particle_layer_default)

Now, let's define a particle layer with a different optical thickness:

In [ ]:
particle_layer_thick = ertsc.atmosphere.ParticleLayer(tau_ref=1.0)
plot_sigma_t(
    particle_layer_default,
    particle_layer_thick,
    labels=["default", "thick"],
)

We can also modify the extent of our particle using the *bottom* and *top* parameters:

In [ ]:
particle_layer_high = ertsc.atmosphere.ParticleLayer(tau_ref=0.5, bottom=1 * ureg.km, top=2 * ureg.km)
plot_sigma_t(
    particle_layer_default,
    particle_layer_thick,
    particle_layer_high,
    labels=["default", "thick", "high"],
)

We can vary the particle density distribution within the defined bounds:

In [ ]:
particle_layer_gaussian = ertsc.atmosphere.ParticleLayer(
    tau_ref=0.5,
    bottom=0.5 * ureg.km,
    top=1.5 * ureg.km,
    distribution="gaussian",
    n_layers=31,
)

plot_sigma_t(
    particle_layer_default,
    particle_layer_thick,
    particle_layer_high,
    particle_layer_gaussian,
    labels=["default", "thick", "high", "gaussian"],
)

In [ ]:
exponential = ertsc.atmosphere.ParticleLayer(
    tau_ref=0.05, 
    bottom=0 * ureg.km, 
    top=6 * ureg.km,
    distribution="exponential",
)

gaussian = ertsc.atmosphere.ParticleLayer(
    tau_ref=0.02, 
    bottom=1 * ureg.km, 
    top=2 * ureg.km,
    distribution="gaussian",
)

atmosphere = ertsc.atmosphere.HeterogeneousAtmosphere(
    molecular_atmosphere={"type": "molecular"},
    particle_layers=[exponential, gaussian]
)

We can display the extinction coefficient for the full atmospheric profile and all its components as follows (note that we restrict the view range to the [0, 10] km extent for clarity

In [ ]:
plot_sigma_t(
    atmosphere,
    atmosphere.molecular_atmosphere,
    exponential,
    gaussian,
    labels=["atmosphere", "molecular", "exponential", "gaussian"],
    altitude_extent=(0, 10),
)

## Running the simulation

We can now use this atmosphere definition to run a 1D simulation and compute the top-of-atmosphere BRF:

In [ ]:
# Show only spectral loop progress
eradiate.config.progress = "spectral_loop"

exp = eradiate.experiments.AtmosphereExperiment(
    surface={"type": "lambertian", "reflectance": 1.0},
    atmosphere=atmosphere,
    illumination={"type": "directional", "zenith": 30.0, "azimuth": 0.0},
    measures={
        "type": "mdistant",
        "construct": "from_viewing_angles",
        "zeniths": np.arange(-75, 76, 5),
        "azimuths": 0.0,
        "spectral_cfg": {"bins": ["550"]},  # Run the simulation for the 550 nm bin
        "spp": 10000,
    },
)
result = eradiate.run(exp)
result

And we can plot the computed TOA BRF:

In [ ]:
with plt.rc_context({"lines.marker": ".", "lines.linestyle": ":"}):
    result.brf.squeeze(drop=True).plot(x="vza");